In [76]:
import json
# json.dump(pos_dict, open("dicts/pos_dict", 'w'))
# json.dump(neg_dict, open("dicts/neg_dict", 'w'))

pos_dict = defaultdict(int, json.load(open('dicts/pos_dict')))

neg_dict = defaultdict(int, json.load(open('dicts/neg_dict')))

In [139]:
def cal_senti_score(pos_tagged_sentences):
    incrementers = ['too', 'very', 'sorely', 'painfully', 'greatly', 'woefully', 'extremely', 'greatly', 'more']
    decrementers = ['barely', 'little', 'less', 'hardly', 'scarcely', 'rarely']
    
    score = []
    
    for sent in pos_tagged_sentences:
        pos_score = []
        neg_score = []
        total = len(sent)
  
        for i, token in enumerate(sent):    
            if pos_dict.get(token[0]) == None:
                pass
            if i == 0:
                pos_score.append(pos_dict[token[0]])
            else:
                previous_token = sent[i-1][0]

                if previous_token in incrementers:
                        pos_score.append(pos_dict[token[0]]*2.)

                elif previous_token in decrementers:

                    pos_score.append(pos_dict[token[0]]/2.)
                else:
                    pos_score.append(pos_dict[token[0]])
                        
        for i, token in enumerate(sent):    
            if neg_dict.get(token[0]) == None:
                pass
            if i == 0:
                neg_score.append(neg_dict[token[0]])
            else:
                previous_token = sent[i-1][0]
    
                if previous_token in incrementers:
                    neg_score.append(neg_dict[token[0]]*2.)

                elif previous_token in decrementers:

                    neg_score.append(neg_dict[token[0]]/2.)
                else:
                    neg_score.append(neg_dict[token[0]])

        sent_score = (sum(pos_score) - sum(neg_score)) / float(total) 
        print pos_score
        print neg_score
    
        score.append(sent_score)
    return sum(score) / float(len(score))

In [140]:
cal_senti_score(pos_tagged_sentences)

[0, 0, 0, 0.0, 0.0, 0, 0.0, 0]
[0, 0, 0, 0.0, 0.0, 0, 0.0, 0]
[0, 0.0, 0, 0, 0.0, 0, 0.0, 0, 0, 0, 0, 0.0, 0.0, 0]
[0, 0.0, 0, 0, 0.0, 0, 0.0, 0, 0, 0, 0, 0.625, 0.625, 0]
[0, 0, 0, 0, 0.25, 0.75, 0, 0, 0, 0, 0.0, 0, 0.0, 1.0, 0]
[0, 0, 0, 0, 0.0, 1.0, 0, 0, 0, 0, 0.0, 0, 0.0, 0.0, 0]
[0, 0, 0.0, 0, 0.0, 0, 0, 0.0, 0.0, 0, 0, 0.0, 0, 0.0, 0, 0.0, 0]
[0, 0, 0.0, 0, 0.375, 0, 0, 0.25, 0.0, 0, 0, 0.0, 0, 0.0, 0, 0.0, 0]
[0, 0.0, 0, 0.0, 0.0, 0, 0.0, 0, 0, 0.5, 0, 0, 0.0, 0]
[0, 0.0, 0, 0.0, 0.0, 0, 0.0, 0, 0, 0.0, 0, 0, 0.0, 0]


-0.00473389355742297

In [5]:
text ="What can I say about this place. \
The staff of the restaurant is nice and the eggplant is not bad. \
Apart from that, very uninspired food, lack of atmosphere and too expensive. \
I am a staunch vegetarian and was sorely dissapointed with the veggie options on the menu. \
Will be the last time I visit, I recommend others to avoid."

Needs to remove stopwords from text, then tag the sentences?
Needs to stem the dictionary, but how?


### TRY PATTERN

In [1]:
from pattern.en import sentiment, polarity, subjectivity
import json
import pandas as pd
from itertools import *

In [37]:
data = pd.read_csv("hilary_text", header = None)
data.drop(0, axis =1, inplace=True)
data.columns = ['comment']
comments = data['comment'].values
#data.sort('comment')

In [35]:
# print len(data.duplicated('comment'))
# print len(comments)
data2 = data.drop_duplicates(['comment'])
np.unique(data['comment'].values)
#print 340./6546

array([ " I take Hilary to be a technocrat, if I'm using the word correctly.  That is, she is someone who gets entrenched in the mechanics of government and works her butt off to get all the details right so that all interested parties get some consideration in the final policy.<br/>   The animosity that has been directed towards her, and the off hand insults are largely unwarranted and stem from identity politics more than anything else.  The woman deserves some respect.  She is a very capable deal maker.<br/>   Unfortunately when you broker deals where everyone wins a little and everyone loses a little, nobody goes away happy least of all those furthest from the center.  But this country needs to be governed from the center and Hilary is the only candidate who can actually do that.  ",
       " She's all for fronting a persona which she and her handlers think will win her the White House. She's something today, but a totally different one tomorrow. I dare any of her supporters to tel

In [9]:
score = []
for i in comments:
    score.append(polarity(i))

KeyboardInterrupt: 

### TRY WORD2VEC

In [1]:
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec
import numpy as numpy
from sklearn.linear_model import LogisticRegression

In [2]:
class LabeledLineSentence(object):
    def __init__(self, sources):
        self.sources = sources

        flipped = {}

        # make sure that keys are unique
        for key, value in sources.items():
            if value not in flipped:
                flipped[value] = [key]
            else:
                raise Exception('Non-unique prefix encountered')

    def __iter__(self):
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                for item_no, line in enumerate(fin):
                    yield LabeledSentence(utils.to_unicode(line).split(), [prefix + '_%s' % item_no])

    def to_array(self):
        self.sentences = []
        for source, prefix in self.sources.items():
            with utils.smart_open(source) as fin:
                for item_no, line in enumerate(fin):
                    self.sentences.append(LabeledSentence(utils.to_unicode(line).split(), [prefix + '_%s' % item_no]))
        return self.sentences

    def sentences_perm(self):
        return numpy.random.permutation(self.sentences)

In [3]:
sources = {'word2vec-sentiments/test-neg.txt':'TEST_NEG', 'word2vec-sentiments/test-pos.txt':'TEST_POS', 
           'word2vec-sentiments/train-neg.txt':'TRAIN_NEG', 'word2vec-sentiments/train-pos.txt':'TRAIN_POS', 
           'word2vec-sentiments/train-unsup.txt':'TRAIN_UNS'}

sentences = LabeledLineSentence(sources)

In [ ]:
import random

model = Doc2Vec(min_count=1, window=10, size=100, sample=1e-4, negative=5, workers=7)
model.build_vocab(sentences.to_array())

sentences_list = sentences.to_array()
idx = range(len(sentences_list))

for epoch in range(10):
    random_idx = random.sample(idx, len(idx))
    perm_sentences = [sentences_list[i] for i in random_idx]
    model.train(perm_sentences)
    # model.train(sentences.sentences_perm())

In [ ]:
train_arrays = np.zeros((25000, 100))
train_labels = np.zeros(25000)

for i in range(12500):
    prefix_train_pos = 'TRAIN_POS_' + str(i)
    prefix_train_neg = 'TRAIN_NEG_' + str(i)
    
    train_arrays[i] = model[prefix_train_pos]
    train_arrays[12500 + i] = model[prefix_train_neg]
    train_labels[i] = 1
    train_labels[12500 + i] = 0

In [ ]:
test_arrays = np.zeros((25000, 100))
test_labels = np.zeros(25000)

for i in range(12500):
    prefix_test_pos = 'TEST_POS_' + str(i)
    prefix_test_neg = 'TEST_NEG_' + str(i)
    test_arrays[i] = model[prefix_test_pos]
    test_arrays[12500 + i] = model[prefix_test_neg]
    test_labels[i] = 1
    test_labels[12500 + i] = 0